In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
%pdb off

from pyCascade import probePost, physics, utils, probeReadWrite
from pyCascade.probeReadWrite import read_probes_file_switch
from filloutVentilationStats import *
from matplotlib import pyplot as plt
from matplotlib import cm, colors
import matplotlib.ticker as ticker
import numpy as np
import scipy as sp
import os
from IPython.core.debugger import set_trace
import pandas as pd
import seaborn as sns
from cycler import cycler
import plotly.express as px
import plotly
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import statsmodels.api as sm
import warnings

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')

## Runs

In [2]:


multiRun_dir = f"{home_dir}/CHARLES/multiRuns/"
plotFolder = f"{multiRun_dir}"

## Read in results

In [ ]:
overWrite = False
if os.path.exists(f"{multiRun_dir}/roomVentilationMI.csv") == False or os.path.exists(f"{multiRun_dir}/flowStatsMI.csv") == False or overWrite:

    runs = {
        246: {'A': 45, 'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        247: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        248: {'A': 0,  'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        249: {'A': 0,  'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        316: {'A': 45, 'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        317: {'A': 45, 'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        318: {'A': 0,  'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        319: {'A': 0,  'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},

        253: {'A': 45, 'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        254: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        255: {'A': 0,  'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        256: {'A': 0,  'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        321: {'A': 45, 'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        322: {'A': 45, 'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        323: {'A': 0,  'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        324: {'A': 0,  'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [120000, 160000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
    }

    flowStatsMI, roomVentilationMI = readRunStats(runs, home_dir, scratch_dir, multiRun_dir = plotFolder, readABLFits = False, saveResults = True)
else:
    flowStatsMI = pd.read_csv(f"{multiRun_dir}/flowStatsMI.csv", index_col = [0,1])
    roomVentilationMI = pd.read_csv(f"{multiRun_dir}/roomVentilationMI.csv", index_col = [0,1])

flowStatsMI = combine_stats(flowStatsMI, ["WS", "delT", "SS", "C", "A"])
roomVentilationMI = combine_stats(roomVentilationMI,   ["WS", "delT", "SS", "C", "A"])
    

## Locations

In [ ]:

# fig, ax=plt.subplots(1, 1, constrained_layout =True)
# ax.scatter(flowStatsMI[flowStatsMI["orientation"] ==   0]['x'], flowStatsMI[flowStatsMI["orientation"] ==   0]['z'], s=4*im_scaling, c="red", edgecolors="black", linewidths=.1)
# ax.scatter(flowStatsMI[flowStatsMI["orientation"] ==  90]['x'], flowStatsMI[flowStatsMI["orientation"] ==  90]['z'], s=4*im_scaling, c="orange", edgecolors="black", linewidths=.1)
# ax.scatter(flowStatsMI[flowStatsMI["orientation"] == 180]['x'], flowStatsMI[flowStatsMI["orientation"] == 180]['z'], s=4*im_scaling, c="blue", edgecolors="black", linewidths=.1)
# ax.scatter(flowStatsMI[flowStatsMI["orientation"] == 270]['x'], flowStatsMI[flowStatsMI["orientation"] == 270]['z'], s=4*im_scaling, c="purple", edgecolors="black", linewidths=.1)

fig = px.scatter_3d(data_frame = flowStatsMI, x = "x", y = flowStatsMI.index.get_level_values(0), z = "z", color = "orientation")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
fig.show()

fig = px.scatter_3d(data_frame = roomVentilationMI, x = "x", y = roomVentilationMI.index.get_level_values(0), z = "z", color = "orientation")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
fig.show()

fig = px.scatter_3d(data_frame = flowStatsMI, x = "x", y = flowStatsMI.index.get_level_values(0), z = "z", color = "roomA")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
fig.show()

plot_df = roomVentilationMI[roomVentilationMI["AofA"] % 90 == 0]
fig = px.scatter_3d(data_frame = plot_df, x = "x", y = plot_df.index.get_level_values(0), z = "z", color = "AofA")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
fig.show()

plot_df = roomVentilationMI[roomVentilationMI["AofA"] % 90 == 45]
fig = px.scatter_3d(data_frame = plot_df, x = "x", y = plot_df.index.get_level_values(0), z = "z", color = "AofA")
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
fig.show()


## Probed Wind Speeds

In [ ]:
windSpeeds = {}

for C in roomVentilationMI["C"].unique():
    windSpeeds[C] = {}
    for WS in roomVentilationMI["WS"].unique():
        windSpeeds[C][WS] = {}
        for A in roomVentilationMI["A"].unique():
            windSpeeds[C][WS][A] = {}
            speeds = roomVentilationMI[(roomVentilationMI["WS"] == WS) & (roomVentilationMI["C"] == C) & (roomVentilationMI["A"] == A)]
            speeds = speeds["EPR_mag"]
            windSpeeds[C][WS][A]["mean"] = np.mean(speeds)

display(windSpeeds)

# Bar Plots


## Comparing QOIS for mass flux

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf["mean-mass_flux(S)-Norm"] =  -plotdf["mean-mass_flux(S)-Norm"]
plotdf[ "mean-sn_prod(abs(u))-Norm"] = rho*plotdf[ "mean-sn_prod(abs(u))-Norm"]
# plotdf = plotdf[plotdf["roomType"] != "single"]
groups = [
    ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType"],
    ["roomType", "WS", "Ri", "delT"],  
]
plotdfGrouped = plotdf.groupby(groups[0], as_index=False).mean(numeric_only=True)

for y_var in ["mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "q-D-room-Norm", "mean-mass_flux(S)-Norm"]:
    x_var = "roomType"
    # Create the bar plot
    plt.figure(figsize=(10, 6))
    g = sns.barplot(data=plotdfGrouped, x=x_var, y=y_var, color="lightgrey", errorbar=None)#("pi", 80))


    # Define a list of colors for the different groups
    PIcolors = ['red', 'green', 'blue', 'purple']
    for i, group in enumerate(groups):
        for j, x_val in enumerate(np.sort(plotdf[x_var].unique())):
            plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
            percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
            mean = percentiles['mean']
            yerr = [[mean - percentiles['10%']], [percentiles['90%'] - mean]]
            plt.errorbar(
                x= j + i/20,
                y=mean,
                yerr=yerr,
                fmt='none',
                ecolor=PIcolors[i],
                capsize=5,
                label=f'Error for group {i + 1}'
            )

    # Adding a legend for the custom error bars (if needed)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
    plt.tick_params(axis='both', which='major', labelsize=16)

## Volume Flux Bar Plots

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
# plotdf = plotdf[plotdf["roomType"] != "single"]
groups = [
    ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType"],
    ["roomType", "WS", "Ri", "delT"],  
]
plotdfGrouped = plotdf.groupby(groups[0], as_index=False).mean(numeric_only=True)

x_var = "roomType"
y_var = "mean-sn_prod(abs(u))-Norm"
# Create the bar plot
plt.figure(figsize=(10, 6))
g = sns.barplot(data=plotdfGrouped, x=x_var, y=y_var, color="lightgrey", errorbar=("pi", 50), capsize = .1)


# # Define a list of colors for the different groups
# PIcolors = ['red', 'green', 'blue', 'purple']
# for i, group in enumerate(groups):
#     for j, x_val in enumerate(np.sort(plotdf[x_var].unique())):
#         plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
#         percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
#         mean = percentiles['mean']
#         yerr = [[mean - percentiles['10%']], [percentiles['90%'] - mean]]
#         plt.errorbar(
#             x= j + i/20,
#             y=mean,
#             yerr=yerr,
#             fmt='none',
#             ecolor=PIcolors[i],
#             capsize=5,
#             label=f'Error for group {i + 1}'
#         )

# Adding a legend for the custom error bars (if needed)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
plt.tick_params(axis='both', which='major', labelsize=16)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
# plotdf = plotdf[plotdf["roomType"] != "single"]
groups = [
    ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType"],
    ["roomType", "WS", "Ri", "delT"],  
]
plotdfGrouped = plotdf.groupby(groups[0], as_index=False).mean(numeric_only=True)

x_var = "roomType"
y_var = "mean-sn_prod(abs(u))-Norm"
# Create the bar plot
plt.figure(figsize=(10, 6))
g = sns.boxplot(data=plotdfGrouped, x=x_var, y=y_var, color="lightgrey")


handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
plt.tick_params(axis='both', which='major', labelsize=16)

# print the median and IQR for each room type
for roomType in plotdfGrouped["roomType"].unique():
    print(f"Room Type: {roomType}")
    stats = plotdfGrouped[plotdfGrouped["roomType"] == roomType]["mean-sn_prod(abs(u))-Norm"].describe(percentiles=[0.25, 0.75])
    IQR = stats['75%'] - stats['25%']
    print(f"Median: {stats['50%']}, IQR: {IQR}, IQR/median: {IQR/stats['50%']}")


In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
# plotdf = plotdf[plotdf["roomType"] != "single"]
groups = [
    ["roomType", "WS", "Ri", "delT", "houseType", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType"],
    ["roomType", "WS", "Ri", "delT"],  
]
plotdfGrouped = plotdf.groupby(["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"], as_index=False).mean(numeric_only=True)

x_var = "roomType"
y_var = "mean-sn_prod(abs(u))-Norm"
# Create the bar plot
plt.figure(figsize=(10, 6))
g = sns.barplot(data=plotdfGrouped, x=x_var, y=y_var, hue="C", palette=["tab:blue", "tab:orange"], errorbar=("pi", 50), capsize = .1)


# # Define a list of colors for the different groups
# PIcolors = ['red', 'blue', 'purple']
# for i, group in enumerate(groups):
#     for j, x_val in enumerate(np.sort(plotdf[x_var].unique())):
#         for k, C in enumerate(plotdf["C"].unique()):
#             plotdfGrouped = plotdf[plotdf["C"]==C].groupby(group, as_index=False).mean(numeric_only=True)
#             percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
#             mean = percentiles['mean']
#             yerr = [[mean - percentiles['10%']], [percentiles['90%'] - mean]]
#             plt.errorbar(
#                 x= j + i/20 + (k - .5)/2,
#                 y=mean,
#                 yerr=yerr,
#                 fmt='none',
#                 ecolor=PIcolors[i],
#                 capsize=5,
#                 label=f'Error for group {i + 1}'
#             )

# Adding a legend for the custom error bars (if needed)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
plt.tick_params(axis='both', which='major', labelsize=16)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
# plotdf = plotdf[plotdf["roomType"] != "single"]
groups = [
    ["roomType", "WS", "Ri", "delT", "houseType", "AofA"],
    ["roomType", "WS", "Ri", "delT", "houseType"],
    ["roomType", "WS", "Ri", "delT"],  
]
plotdfGrouped = plotdf.groupby(["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"], as_index=False).mean(numeric_only=True)

x_var = "roomType"
y_var = "mean-sn_prod(abs(u))-Norm"
# Create the box plot
plt.figure(figsize=(10, 6))
g = sns.boxplot(data=plotdfGrouped, x=x_var, y=y_var, hue="C", palette=["tab:blue", "tab:orange"])

# Adding a legend for the custom error bars (if needed)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
plt.tick_params(axis='both', which='major', labelsize=16)

In [ ]:
# Prepare data for combined box plot
plotdf_all = plotdfGrouped.copy()
plotdf_all["Category"] = "All Ventilation Rates"

plotdf_high_density = plotdfGrouped[plotdfGrouped["C"] == 2].copy()
plotdf_high_density["Category"] = "High Density Canopy"

plotdf_low_density = plotdfGrouped[plotdfGrouped["C"] == 3].copy()
plotdf_low_density["Category"] = "Low Density Canopy"

# Combine all data
plotdf_combined = pd.concat([plotdf_all, plotdf_high_density, plotdf_low_density])

# Create the combined box plot
plt.figure(figsize=(8, 5))
sns.boxplot(data=plotdf_combined, x="roomType", y="mean-sn_prod(abs(u))-Norm", hue="Category", palette=["lightgrey", "tab:blue", "tab:orange"])

# Customize the plot
plt.legend(title="Category", loc='upper right')
plt.tick_params(axis='both', which='major', labelsize=16)
plt.xlabel("Room Type", fontsize=14)
plt.ylabel("Mean SN Prod (abs(u)) Norm", fontsize=14)
plt.title("Combined Box Plot for All Data, High Density, and Low Density", fontsize=16)
plt.tight_layout()
plt.show()

# Print the median and IQR for each room type, high density, and low density
for category, data in [("All Ventilation Rates", plotdfGrouped), 
                       ("High Density", plotdf_high_density), 
                       ("Low Density", plotdf_low_density)]:
    print(f"Category: {category}")
    IQRMean = 0
    medianMean = 0
    for roomType in data["roomType"].unique():
        print(f"  Room Type: {roomType}")
        stats = data[data["roomType"] == roomType]["mean-sn_prod(abs(u))-Norm"].describe(percentiles=[0.25, 0.75])
        IQR = stats['75%'] - stats['25%']
        median = stats['50%']
        print(f"    Median: {median}, IQR: {IQR}, IQR/median: {IQR/median}")
        IQRMean += IQR
        medianMean += median
    IQRMean /= len(data["roomType"].unique())
    medianMean /= len(data["roomType"].unique())
    print(f"  Average Median: {medianMean}, Average IQR: {IQRMean}, Average IQR/median: {IQRMean/medianMean}")
    


In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["roomType"] == "corner"]
groups = [
    ["C", "roomType", "WS", "Ri", "delT", "houseType", "AofA"],
    ["C", "roomType", "WS", "Ri", "delT", "houseType"],
    ["C", "roomType", "WS", "Ri", "delT"],
]
plotdfGrouped = plotdf.groupby(groups[0], as_index=False).mean(numeric_only=True)

x_var = "C"
y_var = "mean-sn_prod(abs(u))-Norm"
# Create the bar plot
plt.figure(figsize=(10, 6))
g = sns.barplot(data=plotdfGrouped, x=x_var, y=y_var, color="lightgrey", errorbar=None)#("pi", 80))


# Define a list of colors for the different groups
PIcolors = ['red', 'blue', 'purple']
for i, group in enumerate(groups):
    for j, x_val in enumerate(np.sort(plotdf[x_var].unique())):
        plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
        percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
        mean = percentiles['mean']
        yerr = [[mean - percentiles['10%']], [percentiles['90%'] - mean]]
        plt.errorbar(
            x= j + i/20,
            y=mean,
            yerr=yerr,
            fmt='none',
            ecolor=PIcolors[i],
            capsize=5,
            label=f'Error for group {i + 1}'
        )

# Adding a legend for the custom error bars (if needed)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
for C_val in [2, 3]:
    plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
    plotdf = plotdf[plotdf["SS"] == True]
    plotdf = plotdf[plotdf["roomType"] == "corner"]
    plotdf = plotdf[plotdf["C"] == C_val]
    groups = [
        ["AofA", "roomType", "WS", "Ri", "delT", "houseType"],
        ["AofA", "roomType", "WS", "Ri", "delT"],
    ]
    plotdfGrouped = plotdf.groupby(groups[0], as_index=False).mean(numeric_only=True)

    x_var = "AofA"
    y_var = "mean-sn_prod(abs(u))-Norm"
    # Create the bar plot
    plt.figure(figsize=(10, 6))
    g = sns.barplot(data=plotdfGrouped, x=x_var, y=y_var, color="lightgrey", errorbar=None)#("pi", 80))


    # Define a list of colors for the different groups
    PIcolors = ['blue', 'purple']
    for i, group in enumerate(groups):
        for j, x_val in enumerate(np.sort(plotdf[x_var].unique())):
            plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
            percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
            mean = percentiles['mean']
            yerr = [[mean - percentiles['10%']], [percentiles['90%'] - mean]]
            plt.errorbar(
                x= j + i/20,
                y=mean,
                yerr=yerr,
                fmt='none',
                ecolor=PIcolors[i],
                capsize=5,
                label=f'Error for group {i + 1}'
            )

    # Adding a legend for the custom error bars (if needed)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
    plt.show()

In [ ]:
r_lim = [.49, .59, .59, .069]
fig, axs = plt.subplots(4, 1, subplot_kw={'projection': 'polar'}, figsize = (10, 16), dpi = 300)
for i, room in enumerate(["corner", "cross", "dual", "single"]):
    plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
    plotdf = plotdf[plotdf["SS"] == True]
    plotdf = plotdf[plotdf["roomType"] == room]
    groups = [
        ["AofA", "roomType", "WS", "Ri", "delT", "houseType", "C"],
        ["AofA", "roomType", "WS", "Ri", "delT", "houseType"],
        ["AofA", "roomType", "WS", "Ri", "delT"],
    ]
    
    x_var = "AofA"
    y_var = "mean-sn_prod(abs(u))-Norm"

    # Create the polar plot
    ax = axs[i]
    ax.set_ylim(0, r_lim[i])
    
    # Customize gridlines to be lighter
    ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
    # Set the radial labels to use scientific notation
    ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.set_rlabel_position(0)

    plotdfGrouped = plotdf.groupby(x_var, as_index=False).mean(numeric_only=True)
    angles = np.deg2rad(plotdfGrouped[x_var])
    radii = plotdfGrouped[y_var].values
    bars = ax.bar(angles, radii, width=np.pi/5, color='lightgrey')
    for ii, group in enumerate(groups):
        plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
        PIcolors = ['red', 'green', 'purple']
        for jj, x_val in enumerate(np.sort(plotdf[x_var].unique())):
            percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
            mean = percentiles['mean']
            yerr = np.array([[mean - percentiles['10%']], [percentiles['90%'] - mean]])
            angle = np.deg2rad(x_val)
            
            ax.errorbar(
                x=angle + ii*np.pi/(400*mean),
                y=mean,
                yerr=yerr,
                fmt='none',
                ecolor=PIcolors[ii],
                capsize=0,
                label=f'Error for group {ii + 1}'
            )
        
    # Adding a legend for the custom error bars
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

    # Customize plot
    ax.set_theta_direction(1)  # counter clockwise
    ax.set_theta_offset(np.pi)  # Starting from the right

plt.tight_layout()

In [ ]:
r_lim = [.49, .59, .59, .069]
fig, axs = plt.subplots(4, 2, subplot_kw={'projection': 'polar'}, figsize = (10, 16), dpi = 300)
for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j, C_val in enumerate([2, 3]):
        plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
        plotdf = plotdf[plotdf["SS"] == True]
        plotdf = plotdf[plotdf["roomType"] == room]
        plotdf = plotdf[plotdf["C"] == C_val]
        groups = [
            ["AofA", "roomType", "WS", "Ri", "delT", "houseType"],
            ["AofA", "roomType", "WS", "Ri", "delT"],
        ]
    
        x_var = "AofA"
        y_var = "mean-sn_prod(abs(u))-Norm"

        # Create the polar plot
        ax = axs[i, j]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        plotdfGrouped = plotdf.groupby(x_var, as_index=False).mean(numeric_only=True)
        angles = np.deg2rad(plotdfGrouped[x_var])
        radii = plotdfGrouped[y_var].values
        bars = ax.bar(angles, radii, width=np.pi/5, color='lightgrey')
        for ii, group in enumerate(groups):
            plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
            PIcolors = ['red', 'purple']
            # for jj, x_val in enumerate(np.sort(plotdf[x_var].unique())):
            #     percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
            #     mean = percentiles['mean']
            #     yerr = np.array([[mean - percentiles['10%']], [percentiles['90%'] - mean]])
            #     angle = np.deg2rad(x_val)
            
            #     ax.errorbar(
            #         x=angle + ii*np.pi/(400*mean),
            #         y=mean,
            #         yerr=yerr,
            #         fmt='none',
            #         ecolor=PIcolors[ii],
            #         capsize=0,
            #         label=f'Error for group {ii + 1}'
            #     )
        
        # Adding a legend for the custom error bars
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])

plt.tight_layout()

In [ ]:

# Define the function to get the corresponding value
def get_table_value(row):

    value_map = {
        (2, 'Aligned'): 1/3,
        (2, 'Staggered'): 3/4,
        (2, 'Diagonal'): 13/24,
        (3, 'Aligned'): 2/7,
        (3, 'Staggered'): 1/2,
        (3, 'Diagonal'): 11/28
    }

    density = row['C']  # Reference column C for density
    angle = row['AofA']     # Reference column A for angle

    # Determine the alignment type using remainders
    if angle % 180 == 0:
        alignment = 'Aligned'
    elif angle % 180 == 90:
        alignment = 'Staggered'
    elif angle % 90 == 45:
        alignment = 'Diagonal'
    else:
        return None  # In case of an unhandled angle

    # Fetch the corresponding value from the table
    return value_map.get((density, alignment))

# Apply the function to fill a new column based on the table
roomVentilationMI['frontAFrac'] = roomVentilationMI.apply(get_table_value, axis=1)

# Display the dataframe
roomVentilationMI['frontAFrac']

# roomVentilationMI["mean-sn_prod(abs(u))-Norm-FAF"] = roomVentilationMI["mean-sn_prod(abs(u))-Norm"] * roomVentilationMI['frontAFrac']


In [ ]:
fig, axs = plt.subplots(1, 2, subplot_kw={'projection': 'polar'}, figsize = (10, 16), dpi = 300)
for j, C_val in enumerate([2, 3]):
    plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
    plotdf = plotdf[plotdf["SS"] == True]
    plotdf = plotdf[plotdf["roomType"] == room]
    plotdf = plotdf[plotdf["C"] == C_val]
    groups = [
        ["AofA", "WS", "Ri", "delT"],
    ]

    x_var = "AofA"
    y_var = "frontAFrac"

    # Create the polar plot
    ax = axs[j]
    ax.set_ylim(0, 1)

    # Customize gridlines to be lighter
    ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
    # Set the radial labels to use scientific notation
    ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.set_rlabel_position(0)

    plotdfGrouped = plotdf.groupby(x_var, as_index=False).mean(numeric_only=True)
    angles = np.deg2rad(plotdfGrouped[x_var])
    radii = plotdfGrouped[y_var].values
    bars = ax.bar(angles, radii, width=np.pi/5, color='lightgrey')

    # Customize plot
    ax.set_theta_direction(1)  # counter clockwise
    ax.set_theta_offset(np.pi)  # Starting from the right

plt.tight_layout()

In [ ]:
r_lim = [.59, .59, .59, .059]
# r_lim = [r/2 for r in r_lim]
fig, axs = plt.subplots(4, 2, subplot_kw={'projection': 'polar'}, figsize = (10, 16), dpi = 300, layout="tight")
safe_colors = px.colors.qualitative.Safe

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j, C_val in enumerate([2, 3]):
        plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
        plotdf = plotdf[plotdf["SS"] == True]
        plotdf = plotdf[plotdf["roomType"] == room]
        plotdf = plotdf[plotdf["C"] == C_val]
        groups = [
            ["AofA", "roomType", "WS", "Ri", "delT"],
        ]
    
        x_var = "AofA"
        y_var = "mean-sn_prod(abs(u))-Norm" #"mean-sn_prod(abs(u))-Norm"

        # Create the polar plot
        ax = axs[i, j]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars
        plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True) #CHANGED TO MEDIAN
        angles = np.deg2rad(plotdfGrouped[x_var])
        radii = plotdfGrouped[y_var].values
        ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth = 0.25) #CHANGED TO MEDIAN
        h_order = ["0--2", "0--1", "0-0", "0-1", "0-2"]
        for k, h in enumerate(h_order):
            width = np.pi/35
            offset = (len(h_order)/2.5 - k)*width

            plotdfHouse = plotdf[plotdf["houseType"] == h]
            plotdfGrouped = plotdfHouse.groupby(x_var, as_index=False).median(numeric_only=True) 
            angles = np.deg2rad(plotdfGrouped[x_var]) + offset
            radii = plotdfGrouped[y_var].values
            # color = "lightgrey"
            color = rgb_to_mpl(safe_colors[k])
            ax.bar(angles, radii, width=width, color=color, edgecolor = 'white', linewidth = .5, alpha = 1)


            # for ii, group in enumerate(groups):
            #     plotdfGrouped = plotdfHouse.groupby(group, as_index=False).mean(numeric_only=True)
            #     PIcolors = ['purple']
            #     for jj, x_val in enumerate(np.sort(plotdf[x_var].unique())):
            #         percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.25, 0.75])
            #         mean = percentiles['mean']
            #         yerr = np.array([[mean - percentiles['25%']], [percentiles['75%'] - mean]])
            #         angle = np.deg2rad(x_val)
            
            #         ax.errorbar(
            #             x=angle + offset + ii*np.pi/(400*mean),
            #             y=mean,
            #             yerr=yerr,
            #             fmt='none',
            #             ecolor="gray", #PIcolors[ii],
            #             capsize=0,
            #             label=f'Error for group {ii + 1}',
            #             alpha = 1,
            #             elinewidth = .8
            #         )
        
        # Adding a legend for the custom error bars
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])

# plt.tight_layout()

In [ ]:
C_val = 3
AofA_val = 0
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["roomType"] == "corner"]
plotdf = plotdf[plotdf["C"] == C_val]
plotdf = plotdf[plotdf["AofA"] == AofA_val]
groups = [
    ["houseType", "roomType", "WS", "Ri", "delT"],
]
plotdfGrouped = plotdf.groupby(groups[0], as_index=False).mean(numeric_only=True)

x_var = "houseType"
y_var = "mean-sn_prod(abs(u))-Norm"
# Create the bar plot
plt.figure(figsize=(10, 6))
h_order = ["0--2", "0--1", "0-0", "0-1", "0-2"]
g = sns.barplot(data=plotdfGrouped, x=y_var, y=x_var, color="lightgrey", orient = "h", errorbar=None, order = h_order)#("pi", 80))


# Define a list of colors for the different groups
PIcolors = ['purple']
for i, group in enumerate(groups):
    for j, x_val in enumerate(h_order):
        plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
        percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
        mean = percentiles['mean']
        yerr = [[mean - percentiles['10%']], [percentiles['90%'] - mean]]
        plt.errorbar(
            x=mean,
            y= j + i/20,
            xerr=yerr,
            fmt='none',
            ecolor=PIcolors[i],
            capsize=5,
            label=f'Error for group {i + 1}'
        )

# Adding a legend for the custom error bars (if needed)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
C_val = 3
AofA_val = 315
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["roomType"] == "corner"]
plotdf = plotdf[plotdf["C"] == C_val]
plotdf = plotdf[plotdf["AofA"] == AofA_val]
groups = [
    ["houseType", "roomType", "WS", "Ri", "delT"],
]
plotdfGrouped = plotdf.groupby(groups[0], as_index=False).mean(numeric_only=True)

x_var = "houseType"
y_var = "mean-sn_prod(abs(u))-Norm"
# Create the bar plot
plt.figure(figsize=(10, 6))
h_order = ["0--2", "0--1", "0-0", "0-1", "0-2"]
g = sns.barplot(data=plotdfGrouped, x=x_var, y=y_var, color="lightgrey", errorbar=None, order = h_order)#("pi", 80))


# Define a list of colors for the different groups
PIcolors = ['purple']
for i, group in enumerate(groups):
    for j, x_val in enumerate(h_order):
        plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)
        percentiles = plotdfGrouped[plotdfGrouped[x_var] == x_val][y_var].describe(percentiles=[0.1, 0.9])
        mean = percentiles['mean']
        yerr = [[mean - percentiles['10%']], [percentiles['90%'] - mean]]
        plt.errorbar(
            x= j + i/20,
            y=mean,
            yerr=yerr,
            fmt='none',
            ecolor=PIcolors[i],
            capsize=5,
            label=f'Error for group {i + 1}'
        )

# Adding a legend for the custom error bars (if needed)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))
plt.gca().invert_xaxis()
plt.show()

In [ ]:
g = sns.FacetGrid(flowStatsMI[flowStatsMI["roomA"] == 90], col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "meanNorm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
# g.map(sns.boxplot, "net", "roomType", color = "1", linewidth = 1, fliersize = 0)
g.map(sns.swarmplot, "mean-mass_flux-Norm", "houseType", "openingType", palette = "tab10", size = 4, hue_order = ["zwindow", "xwindow", "skylight"], order = ["3-0", "3-1", "3-2", "3-3", "3-4", "sl"])
g.add_legend()

In [ ]:
plotdf = flowStatsMI[flowStatsMI["AofA"]%1 == 0]
plotdf = plotdf[plotdf["SS"] == True]

fig = px.scatter(data_frame = plotdf, x = "EPR_mag", y = "net-mass_flux", color = "roomType", symbol="roomType")
fig.update_xaxes(title_text='2H Wind Speed')
fig.update_yaxes(title_text='Normalized Ventilation Rate')
# fig.update_coloraxes(title_text='Local Wind Magnitude')
fig.show()
 
fig = px.scatter(data_frame = plotdf, x = "EP_normal", y = "mean-mass_flux",  color = "EP_shear", symbol = "roomType", hover_data=["AofA", "windowType", "EP_vel_orientation"])
fig.update_xaxes(title_text='Component of Wind Normal to Window')
fig.update_yaxes(title_text='Normalized Ventilation Rate')
fig.update_layout(
    legend=dict(x=0, y=1.1)  # Adjust the legend position using x and y coordinates
)
# fig.update_coloraxes(title_text='Local Wind Magnitude')
fig.show()


In [ ]:

fig = px.scatter_3d(data_frame = plotdf, x = "EP_shear", y = "EP_normal", z = "mean-mass_flux", color = "roomType", symbol = "windowType", hover_data=["AofA", "windowType"])
fig.update_xaxes(title_text='Component of Wind Normal to Window')
fig.update_yaxes(title_text='Normalized Ventilation Rate')
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
# fig.update_coloraxes(title_text='Local Wind Magnitude')
fig.show()

fig = px.scatter_3d(data_frame = plotdf, x = np.abs(plotdf["EP_vel_orientation"]-90), y = "EP_normal", z = "mean-mass_flux", color = "roomType", symbol = "windowType", hover_data=["AofA", "windowType"])
fig.update_xaxes(title_text='Component of Wind Normal to Window')
fig.update_yaxes(title_text='Normalized Ventilation Rate')
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
# fig.update_coloraxes(title_text='Local Wind Magnitude')
fig.show()

plotdf = roomVentilationMI[roomVentilationMI["AofA"]%1 == 0]
plotdf = plotdf[plotdf["SS"] == True]

fig = px.scatter(data_frame = plotdf, x = "EPR_mag", y = "net-mass_flux", color = "roomType", symbol="roomType")
fig.update_xaxes(title_text='2H Wind Speed')
fig.update_yaxes(title_text='Normalized Ventilation Rate')
# fig.update_coloraxes(title_text='Local Wind Magnitude')
fig.show()
 
fig = px.scatter_3d(data_frame = plotdf, x = "EP_normal_mag", y = "EP_normal_sum",  z = "net-mass_flux", color = "roomType", symbol = "roomType")
fig.update_xaxes(title_text='Magnitude/Sum of Window Normal Velocities')
fig.update_yaxes(title_text='Normalized Ventilation Rate')
# fig.update_coloraxes(title_text='Local Wind Magnitude')
fig.update_traces(marker=dict(size=3))
fig.update_layout(width=800, height=600)  # Set width and height as per your preference
fig.show()

In [ ]:
plotdf = flowStatsMI[flowStatsMI["AofA"]%1 == 0]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["houseType"] != "sl"]
g = sns.FacetGrid(plotdf, col = "windowType", col_wrap = 2, aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "mean-mass_flux-Norm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
g.map(sns.lineplot, "AofA", "mean-mass_flux-Norm", "Params", palette = "tab10")
g.add_legend()

g = sns.FacetGrid(plotdf, col = "windowType", col_wrap = 2, aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "mean-mass_flux-Norm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
g.map(sns.lineplot, "AofA", "mean-mass_flux-Norm", "Params", palette = "tab10")
g.add_legend()

g = sns.FacetGrid(plotdf, col = "windowType", col_wrap = 2, aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "mean-mass_flux-Norm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
g.map(sns.lineplot, "AofA", "mean-mass_flux-NormEP", "Params", palette = "tab10")
g.add_legend()

In [ ]:
plotdfGrouped = plotdf.groupby(["roomType", "houseType", "Params", "AofA"], as_index=False).mean(numeric_only = True)
g = sns.FacetGrid(plotdfGrouped, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "mean-mass_flux-Norm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
g.map(sns.lineplot, "AofA", "mean-mass_flux-Norm", "Params", palette = "tab10")
g.add_legend()

plotdfGrouped = plotdf.groupby(["roomType", "houseType", "Params", "AofA"], as_index=False).mean(numeric_only = True)
g = sns.FacetGrid(plotdfGrouped, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "mean-mass_flux-Norm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
g.map(sns.lineplot, "AofA", "mean-mass_flux-Norm", "Params", palette = "tab10")
g.add_legend()

plotdfGrouped = plotdf.groupby(["roomType", "houseType", "Params", "AofA"], as_index=False).mean(numeric_only = True)
g = sns.FacetGrid(plotdfGrouped, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "mean-mass_flux-Norm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
g.map(sns.lineplot, "AofA", "mean-mass_flux-NormEP", "Params", palette = "tab10")
g.add_legend()

In [ ]:
g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
# g.map(sns.violinplot, "mean-mass_flux-Norm", "houseType", color = "1", inner = None, linewidth = 1, density_norm = "width", cut = 0)
g.map(sns.lineplot, "AofA", "mean-mass_flux-Norm", "C", palette = "tab10", errorbar = ("pi", 80))
g.add_legend()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf["AofA_str"] = plotdf["AofA"].astype("string")
g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
# g.map(sns.scatterplot, "mean-mass_flux-Norm", "AofA_str", "B", style = roomVentilationMI["WS"], palette = "tab10", markers = [10, 11])
# g.map(sns.stripplot, "mean-mass_flux-Norm", "AofA_str", "Params", palette = "tab10", size=2, dodge = True, jitter= False)
g.map(sns.violinplot, "mean-mass_flux-Norm", "AofA_str", "Params", fill = False, inner = "stick", linewidth = .5, density_norm = "width", cut = 0, split=True)

# g.map(sns.rugplot, "mean-mass_flux-Norm", "AofA", "B")
g.add_legend()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.sort_values(["Params"])
fig1 = px.bar_polar(plotdf, r="mean-mass_flux-Norm", theta="AofA",
                   color="roomType", template="plotly_dark", pattern_shape = "WS",
                   color_discrete_sequence= px.colors.sequential.Viridis)
fig1.update_layout(polar=dict(angularaxis=dict(rotation=180, direction="counterclockwise")))
fig1.show()

fig2 = px.bar_polar(plotdf, r="mean-mass_flux-Norm", theta="AofA",
                   color="roomType", template="plotly_dark", pattern_shape = "delT",
                   color_discrete_sequence= px.colors.sequential.Viridis)
fig2.update_layout(polar=dict(angularaxis=dict(rotation=180, direction="counterclockwise")))
fig2.show()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.sort_values(["houseType", "Params"])
# plotdf = plotdf.sort_values("Params")
color_labels = ["Params"]
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    fracN = plotdf_frac.shape[0] / (len(set(plotdf_frac["roomType"])) * len(set(plotdf_frac["AofA"])))
    plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm"]] = plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm"]] / fracN
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        fig = px.bar_polar(plotdf_frac[plotdf_frac["roomType"] == room], r="mean-mass_flux-Norm", theta="AofA",
                    color=color_label, template="plotly_dark", hover_data = ["houseType"],
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180))) # This only is updating the first subplot


    fig.show()
    fig.write_image(f'{plotFolder}radialByParams.png', scale=2)
    
    # print(fig)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.groupby(["roomType", "AofA"], as_index=False).mean(numeric_only = True)
# plotdf = plotdf.sort_values(["houseType", "Params"])
# plotdf = plotdf.sort_values("Params")
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    fracN = plotdf_frac.shape[0] / (len(set(plotdf_frac["roomType"])) * len(set(plotdf_frac["AofA"])))
    plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm", "net-mass_flux-Norm", "mean-mass_flux-Norm"]] = plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm", "net-mass_flux-Norm", "mean-mass_flux-Norm"]] / fracN
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        plotdf_frac_room = plotdf_frac[plotdf_frac["roomType"] == room]
        fig = px.bar_polar(plotdf_frac_room, r="mean-mass_flux-Norm", theta="AofA",
                    template="plotly_dark",
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.5])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180)))


    fig.show()
    fig.write_image(f'{plotFolder}radialByParamsEPR.png', scale=2)
    
    # print(fig)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.groupby(["roomType", "AofA","C"], as_index=False).mean(numeric_only = True)
# plotdf = plotdf.sort_values(["Params"])
# plotdf = plotdf.sort_values("Params")
color_labels = ["C"]
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    fracN = plotdf_frac.shape[0] / (len(set(plotdf_frac["roomType"])) * len(set(plotdf_frac["AofA"])))
    plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm"]] = plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm"]] / fracN
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        fig = px.bar_polar(plotdf_frac[plotdf_frac["roomType"] == room], r="mean-mass_flux-Norm", theta="AofA",
                    color=color_label, template="plotly_dark",
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180)))


    fig.show()
    fig.write_image(f'{plotFolder}radialByParams.png', scale=2)
    
    # print(fig)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["C"] == 3]
plotdf = plotdf.groupby(["roomType", "AofA"], as_index=False).mean(numeric_only = True)
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    plotdf_frac = pd.concat([plotdf_frac, plotdf_frac[plotdf_frac["AofA"] == 0]], axis = "index")
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        fig = px.line_polar(plotdf_frac[plotdf_frac["roomType"] == room], r="mean-sn_prod(abs(u))-Norm", theta="AofA",
                    template="plotly_dark", hover_data = ["AofA"],
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.6])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180)))


    fig.show()
    fig.write_image(f'{plotFolder}radialByParams.png', scale=2)
    
    # print(fig)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.groupby(["roomType", "C", "AofA"], as_index=False).mean(numeric_only = True)
plotdf = plotdf.sort_values(["AofA", "C"])
# plotdf = plotdf.sort_values("Params")
color_labels = ["C"]
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    plotdf_frac = pd.concat([plotdf_frac, plotdf_frac[plotdf_frac["AofA"] == 0]], axis = "index")
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        fig = px.line_polar(plotdf_frac[plotdf_frac["roomType"] == room], r="mean-sn_prod(abs(u))-Norm", theta="AofA",
                    color=color_label, template="plotly_dark", hover_data = ["AofA"],
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.6])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180)))


    fig.show()
    fig.write_image(f'{plotFolder}radialByParams.png', scale=2)
    
    # print(fig)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["C"] == 2]
plotdf = plotdf[plotdf["WS"] == 4]
plotdf = plotdf.groupby(["roomType", "Ri", "AofA"], as_index=False).mean(numeric_only = True)
plotdf = plotdf.sort_values(["AofA", "Ri"])
# plotdf = plotdf.sort_values("Params")
color_labels = ["Ri"]
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    plotdf_frac = pd.concat([plotdf_frac, plotdf_frac[plotdf_frac["AofA"] == 0]], axis = "index")
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        fig = px.line_polar(plotdf_frac[plotdf_frac["roomType"] == room], r="mean-sn_prod(abs(u))-Norm", theta="AofA",
                    color=color_label, template="plotly_dark", hover_data = ["AofA"],
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.6])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180)))


    fig.show()
    fig.write_image(f'{plotFolder}radialByParams.png', scale=2)
    
    # print(fig)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["C"] == 3]
plotdf = plotdf.groupby(["roomType", "houseType", "C", "AofA"], as_index=False).mean(numeric_only = True)
plotdf = plotdf.sort_values(["AofA", "houseType"])
# plotdf = plotdf.sort_values("Params")
color_labels = [["houseType"]]
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    plotdf_frac["colorCol"] = plotdf_frac[color_label].apply(lambda row: '-'.join([str(val) for val in row]), axis=1)
    plotdf_frac = pd.concat([plotdf_frac, plotdf_frac[plotdf_frac["AofA"] == 0]], axis = "index")
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        fig = px.line_polar(plotdf_frac[plotdf_frac["roomType"] == room], r="mean-sn_prod(abs(u))-Norm", theta="AofA",
                    color="colorCol", template="plotly_dark", hover_data = ["AofA"],
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.6])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180)))


    fig.show()
    fig.write_image(f'{plotFolder}radialByParams.png', scale=2)
    
    # print(fig)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.sort_values(["houseType", "Params"])
# plotdf = plotdf.sort_values("Params")
color_labels = ["Params"]
for i, color_label in enumerate(color_labels):
    plotdf_frac = plotdf.copy()
    fracN = plotdf_frac.shape[0] / (len(set(plotdf_frac["roomType"])) * len(set(plotdf_frac["AofA"])))
    plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm", "net-mass_flux-Norm", "mean-mass_flux-Norm"]] = plotdf_frac[["net-mass_flux-Norm", "mean-mass_flux-Norm", "net-mass_flux-Norm", "mean-mass_flux-Norm"]] / fracN
    rooms = ["corner", "single", "dual", "cross"]
    # params = set(plotdf[color_label])
    # marker_color=px.colors.qualitative.Safe #["#E4FF87", '#709BFF', '#709BFF', '#FFAA70', '#FFAA70', '#FFDF70', '#B6FFB4']
    # color_map = {}
    # for i, param in enumerate(params):
    #     color_map[i] = marker_color[i]
    # color_map = px.colors.sequential.Viridis

    figures = []
    for i, room in enumerate(rooms):
        plotdf_frac_room = plotdf_frac[plotdf_frac["roomType"] == room]
        fig = px.bar_polar(plotdf_frac_room, r="mean-mass_flux-Norm", theta="AofA",
                    color=color_labels[0], template="plotly_dark", hover_data = ["houseType"],
                    color_discrete_sequence=px.colors.qualitative.Safe, title=room)
        figures.append(fig)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2, subplot_titles=rooms, vertical_spacing=0.2)

    for i, figure in enumerate(figures):
        for trace in range(len(figure["data"])):
            room = rooms[i]
            row = int((i/2)+1)
            col = int((i%2)+1)
            # print(row, col)
            trace = figure["data"][trace]
            if i>0:
                trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)
            fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

    # fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.5])
    fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.5])
    fig.update_layout(width = 1000, height = 800, polar=dict(angularaxis=dict(rotation=180)))


    fig.show()
    fig.write_image(f'{plotFolder}radialByParamsEPR.png', scale=2)
    
    # print(fig)

In [ ]:
for AofA_val in [0, 315]:
    plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
    plotdf = plotdf[plotdf["SS"] == True]
    plotdf = plotdf[plotdf["C"] == 3]
    plotdf = plotdf[plotdf["AofA"] == AofA_val]
    plotdf = plotdf[plotdf["roomType"] == "corner"]
    plotdf = plotdf.groupby(["roomType", "houseType", "AofA"], as_index=False).mean(numeric_only = True)
    # fig, ax=plt.subplots(1, 1, constrained_layout =True)
    # im=ax.scatter(plotdf['x'], plotdf['z'], s=4*im_scaling, c=plotdf["mean-sn_prod(abs(u))-Norm"], cmap='binary', edgecolors="black", linewidths=.1)
    # fig.colorbar(im)
    fig = px.scatter(data_frame = plotdf, x = 'x', y = 'z', color = "mean-sn_prod(abs(u))-Norm", hover_data = ['roomType', 'houseType'])  # Set the color range limits)
    fig.update_traces(marker=dict(
        size=32  # Set the size of the markers
    ))
    fig.update_layout(
        xaxis=dict(
            scaleanchor="y",
            scaleratio=1,
        ),
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            # autorange="reversed"  # Reverse the z-axis to create a left-handed coordinate system
        ),
        width=850,  # You can adjust the width and height as needed
        height=800,
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white'  # Set paper background to white
    )
    fig.show()


In [ ]:
for remA in [0, 45]:
    plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
    plotdf = plotdf[plotdf["SS"] == True]
    plotdf = plotdf[plotdf["C"] == 3]
    plotdf = plotdf[plotdf["AofA"] % 90 == remA]
    plotdf = plotdf[plotdf["roomType"] == "corner"]
    plotdf = plotdf.groupby(["roomType", "houseType", "AofA"], as_index=False).mean(numeric_only = True)
    # fig, ax=plt.subplots(1, 1, constrained_layout =True)
    # im=ax.scatter(plotdf['x'], plotdf['z'], s=4*im_scaling, c=plotdf["mean-sn_prod(abs(u))-Norm"], cmap='binary', edgecolors="black", linewidths=.1)
    # fig.colorbar(im)
    fig = px.scatter(data_frame = plotdf, x = 'x', y = 'z', color = "mean-sn_prod(abs(u))-Norm", hover_data = ['roomType', 'houseType'], range_color=[0, 0.6])  # Set the color range limits)
    fig.update_traces(marker=dict(
        size=10  # Set the size of the markers
    ))
    fig.update_layout(
        xaxis=dict(
            scaleanchor="y",
            scaleratio=1,
        ),
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            # autorange="reversed"  # Reverse the z-axis to create a left-handed coordinate system
        ),
        width=850,  # You can adjust the width and height as needed
        height=800,
        plot_bgcolor='white',  # Set plot background to white
        paper_bgcolor='white'  # Set paper background to white
    )
    fig.show()



In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]

for CValue in set(plotdf['C'].values):
    dfCompWS, valuesWS = getComparativeDf(plotdf[plotdf["C"] == CValue], ["WS"])
    fig = px.scatter(x = dfCompWS[('net-mass_flux-Norm', *valuesWS[0])], y = dfCompWS[('net-mass_flux-Norm', *valuesWS[1])], symbol = indMulti(dfCompWS.index, ['C']), color = dfCompWS[('Ri', *valuesWS[0])], hover_data = indMulti(dfCompWS.index, ['AofA', 'C']))

    fig.add_shape(type="line",
                x0=0, 
                y0=0, 
                x1=np.max(dfCompWS["net-mass_flux-Norm"]), 
                y1=np.max(dfCompWS["net-mass_flux-Norm"]))

    fig.update_layout(
        # xaxis_title=r'$\text{Normalized Ventilation Rate} V_{10) = 2 m/s$',
        xaxis_title=r'$U_{10} = 2 m/s, \text{Normalized Ventilation Rate } [\frac{Q}{U_{10}A}]$',
        yaxis_title=r'$U_{10} = 4 m/s, \text{Normalized Ventilation Rate } [\frac{Q}{U_{10}A}]$',
        # legend_title=r'Room Type',
    )
    fig.show()
    fig.write_image(f'{plotFolder}scatterWS_2-4.png', scale=2)


    # fig = px.scatter(x = dfCompWS[('net-mass_flux-Norm', *valuesWS[1])], y = dfCompWS[('net-mass_flux-Norm', *valuesWS[2])], symbol = indMulti(dfCompWS.index, ['roomType']), color = indMulti(dfCompWS.index, ['Ri']), hover_data = indMulti(dfCompWS.index, ['Ri, 'AofA']))

    # fig.add_shape(type="line",
    #               x0=0, 
    #               y0=0, 
    #               x1=np.max(dfCompWS["net-mass_flux-Norm"]), 
    #               y1=np.max(dfCompWS["net-mass_flux-Norm"]))

    # fig.update_layout(
    #     # xaxis_title=r'$\text{Normalized Ventilation Rate} V_{10) = 2 m/s$',
    #     xaxis_title=r'$U_{10} = 4 m/s, \text{Normalized Ventilation Rate } [\frac{Q}{U_{10}A}]$',
    #     yaxis_title=r'$U_{10} = 6 m/s, \text{Normalized Ventilation Rate } [\frac{Q}{U_{10}A}]$',
    #     # legend_title=r'Room Type',
    # )
    # fig.show()
    # fig.write_image(f'{plotFolder}scatterWS_4-6.png', scale=2)

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
# plotdf = plotdf.sample(frac=1)
# plotdf = plotdf.sort_values(["Params"])
plotdf = plotdf.sort_values(["Params", "houseType"])

categorizeBy = ["C", "roomType"]
categoryValues = [plotdf[category].unique() for category in categorizeBy]
splitAlong = ["WS"]
splitInd = [0, 1]
figures = []
qoi = "mean-sn_prod(abs(u))-Norm"
for i, v0 in enumerate(categoryValues[0]):
    for j, v1 in enumerate(categoryValues[1]):
        plotdfSplit = plotdf[((plotdf[categorizeBy[0]] == v0) & (plotdf[categorizeBy[1]] == v1))]
        dfComp, values = getComparativeDf(plotdfSplit, splitAlong)
        color = dfComp[('A', *values[0])]
        color = color.astype(str)
        fig = px.scatter(x = dfComp[qoi].iloc[:,splitInd[0]], y = dfComp[qoi].iloc[:,splitInd[1]], 
            color = color, symbol = indMulti(dfComp.index, ['delT']),  hover_data = indMulti(dfComp.index, ['C', 'roomType', 'AofA']),
            symbol_sequence=['circle', 'triangle-up'])
        figures.append(fig)

num_rows = len(categoryValues[0])
num_cols = len(categoryValues[1])

titleText = []
for CValue in categoryValues[1]:
    text = f"{categorizeBy[1]} = "
    if isinstance(CValue, float):
        text = f"{text}{np.round(CValue, 2)}"
    else:
        text = f"{text}{CValue}"
    titleText.append(text)

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)#, subplot_titles=titleText)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        row = int(i / num_cols) + 1  # Calculate row index
        col = (i % num_cols) + 1  # Calculate column index
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800, title_text = f"{splitAlong[0]} {values[splitInd[0]][0]} vs {values[splitInd[1]][0]}")
fig.update_coloraxes(colorscale = "mrybm", colorbar_orientation = "h", colorbar_y = -0.2, cmin = 0, cmax = 360)
fig.update_xaxes(tickfont=dict(size=16))
fig.update_yaxes(tickfont=dict(size=16))

max_value = np.max(plotdf[qoi]) * 1.2
fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value, line=dict(color="Black"), row="all", col="all")

fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value * 0.75, 
              line=dict(color="Black", dash="dash"), row="all", col="all")
fig.add_shape(type="line", x0=0, y0=0, x1=max_value * 0.75, y1=max_value, 
              line=dict(color="Black", dash="dash"), row="all", col="all")


# Update x-axis and y-axis properties for each subplot
for i, v0 in enumerate(categoryValues[0]):
    yLabelText = f"{categorizeBy[0]} = {v0}"
    fig.update_yaxes(title_text=yLabelText, row=i+1, col=1)  # Add y-axis label

for j, v1 in enumerate(categoryValues[1]):
    xLabelText = f"{categorizeBy[1]} = {v1}"
    fig.update_xaxes(title_text=xLabelText, row=num_rows, col=j+1)  # Add x-axis label


fig.show()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
# plotdf = plotdf.sample(frac=1)
# plotdf = plotdf.sort_values(["Params"])
plotdf = plotdf.sort_values(["Params", "houseType"])

categorizeBy = ["C", "roomType"]
categoryValues = [plotdf[category].unique() for category in categorizeBy]
splitAlong = ["SS"]
splitInd = [0, 1]
qoi = "q-D-room-Norm"
figures = []
for i, v0 in enumerate(categoryValues[0]):
    for j, v1 in enumerate(categoryValues[1]):
        plotdfSplit = plotdf[((plotdf[categorizeBy[0]] == v0) & (plotdf[categorizeBy[1]] == v1))]
        dfComp, values = getComparativeDf(plotdfSplit, splitAlong)
        color = dfComp[('A', *values[0])]
        color = color.astype(str)
        fig = px.scatter(x = dfComp[qoi].iloc[:,splitInd[0]], y = dfComp[qoi].iloc[:,splitInd[1]], 
            color = color, symbol = indMulti(dfComp.index, ['WS']),  hover_data = indMulti(dfComp.index, ['C', 'roomType', 'AofA']),
            symbol_sequence=['circle', 'triangle-up'])
        figures.append(fig)

num_rows = len(categoryValues[0])
num_cols = len(categoryValues[1])

titleText = []
for CValue in categoryValues[1]:
    text = f"{categorizeBy[1]} = "
    if isinstance(CValue, float):
        text = f"{text}{np.round(CValue, 2)}"
    else:
        text = f"{text}{CValue}"
    titleText.append(text)

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)#, subplot_titles=titleText)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        row = int(i / num_cols) + 1  # Calculate row index
        col = (i % num_cols) + 1  # Calculate column index
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800, title_text = f"{splitAlong[0]} {values[splitInd[0]][0]} vs {values[splitInd[1]][0]}")
fig.update_coloraxes(colorscale = "mrybm", colorbar_orientation = "h", colorbar_y = -0.2, cmin = 0, cmax = 360)

max_value = np.max(plotdf[qoi]) * 1.2
fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value, line=dict(color="Black"), row="all", col="all")

fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value * 0.75, 
              line=dict(color="Black", dash="dash"), row="all", col="all")
fig.add_shape(type="line", x0=0, y0=0, x1=max_value * 0.75, y1=max_value, 
              line=dict(color="Black", dash="dash"), row="all", col="all")


# Update x-axis and y-axis properties for each subplot
for i, v0 in enumerate(categoryValues[0]):
    yLabelText = f"{categorizeBy[0]} = {v0}"
    fig.update_yaxes(title_text=yLabelText, row=i+1, col=1)  # Add y-axis label

for j, v1 in enumerate(categoryValues[1]):
    xLabelText = f"{categorizeBy[1]} = {v1}"
    fig.update_xaxes(title_text=xLabelText, row=num_rows, col=j+1)  # Add x-axis label

fig.show()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
# plotdf = plotdf.sample(frac=1)
# plotdf = plotdf.sort_values(["Params"])
plotdf = plotdf.sort_values(["Params", "houseType"])

categorizeBy = ["C", "roomType"]
categoryValues = [plotdf[category].unique() for category in categorizeBy]
splitAlong = ["delT"]
splitInd = [0, 1]
figures = []
qoi = "mean-sn_prod(abs(u))-Norm"
for i, v0 in enumerate(categoryValues[0]):
    for j, v1 in enumerate(categoryValues[1]):
        plotdfSplit = plotdf[((plotdf[categorizeBy[0]] == v0) & (plotdf[categorizeBy[1]] == v1))]
        dfComp, values = getComparativeDf(plotdfSplit, splitAlong)
        color = dfComp[('A', *values[0])]
        color = color.astype(str)
        fig = px.scatter(x = dfComp[qoi].iloc[:,splitInd[0]], y = dfComp[qoi].iloc[:,splitInd[1]], 
            color = color, symbol = indMulti(dfComp.index, ['WS']),  hover_data = indMulti(dfComp.index, ['C', 'roomType', 'AofA']),
            symbol_sequence=['circle', 'triangle-up'])
        figures.append(fig)

num_rows = len(categoryValues[0])
num_cols = len(categoryValues[1])

titleText = []
for CValue in categoryValues[1]:
    text = f"{categorizeBy[1]} = "
    if isinstance(CValue, float):
        text = f"{text}{np.round(CValue, 2)}"
    else:
        text = f"{text}{CValue}"
    titleText.append(text)

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)#, subplot_titles=titleText)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        row = int(i / num_cols) + 1  # Calculate row index
        col = (i % num_cols) + 1  # Calculate column index
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800, title_text = f"{splitAlong[0]} {values[splitInd[0]][0]} vs {values[splitInd[1]][0]}")
fig.update_coloraxes(colorscale = "mrybm", colorbar_orientation = "h", colorbar_y = -0.2, cmin = 0, cmax = 360)
fig.update_xaxes(tickfont=dict(size=16))
fig.update_yaxes(tickfont=dict(size=16))

max_value = np.max(plotdf[qoi]) * 1.2
fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value, line=dict(color="Black"), row="all", col="all")

fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value * 0.75, 
              line=dict(color="Black", dash="dash"), row="all", col="all")
fig.add_shape(type="line", x0=0, y0=0, x1=max_value * 0.75, y1=max_value, 
              line=dict(color="Black", dash="dash"), row="all", col="all")


# Update x-axis and y-axis properties for each subplot
for i, v0 in enumerate(categoryValues[0]):
    yLabelText = f"{categorizeBy[0]} = {v0}"
    fig.update_yaxes(title_text=yLabelText, row=i+1, col=1)  # Add y-axis label

for j, v1 in enumerate(categoryValues[1]):
    xLabelText = f"{categorizeBy[1]} = {v1}"
    fig.update_xaxes(title_text=xLabelText, row=num_rows, col=j+1)  # Add x-axis label

fig.show()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.sort_values(["Params", "houseType"])

categorizeBy = ["C", "roomType"]
categoryValues = [plotdf[category].unique() for category in categorizeBy]
splitAlong = ["Ri"]
splitInd = [0, 1]
figures = []
for i, v0 in enumerate(categoryValues[0]):
    for j, v1 in enumerate(categoryValues[1]):
        plotdfSplit = plotdf[((plotdf[categorizeBy[0]] == v0) & (plotdf[categorizeBy[1]] == v1))]
        dfComp, values = getComparativeDf(plotdfSplit, splitAlong, sort_cols = ['WS', 'AofA', 'C', 'Ri', 'roomType', 'houseType', 'blockType'])
        color = indMulti(dfComp.index, ['AofA']).astype(float)
        fig = px.scatter(x = dfComp['mean-sn_prod(abs(u))-Norm'].iloc[:,splitInd[0]], y = dfComp['mean-sn_prod(abs(u))-Norm'].iloc[:,splitInd[1]], 
            color = color, symbol = indMulti(dfComp.index, ['WS']),  hover_data = indMulti(dfComp.index, ['C', 'roomType', 'AofA', 'houseType']))
        figures.append(fig)

num_rows = len(categoryValues[0])
num_cols = len(categoryValues[1])

titleText = []
for CValue in categoryValues[1]:
    text = f"{categorizeBy[1]} = "
    if isinstance(CValue, float):
        text = f"{text}{np.round(CValue, 2)}"
    else:
        text = f"{text}{CValue}"
    titleText.append(text)

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)#, subplot_titles=titleText)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        row = int(i / num_cols) + 1  # Calculate row index
        col = (i % num_cols) + 1  # Calculate column index
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800, title_text = f"{splitAlong[0]} {values[splitInd[0]][0]} vs {values[splitInd[1]][0]}")
fig.update_coloraxes(colorscale = "mrybm", colorbar_orientation = "h", colorbar_y = -0.2, cmin = 0, cmax = 360)

max_value = np.max(plotdf["mean-sn_prod(abs(u))-Norm"]) * 1.2
fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value, line=dict(color="Black"), row="all", col="all")

fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value * 0.75, 
              line=dict(color="Black", dash="dash"), row="all", col="all")
fig.add_shape(type="line", x0=0, y0=0, x1=max_value * 0.75, y1=max_value, 
              line=dict(color="Black", dash="dash"), row="all", col="all")
# Update x-axis and y-axis properties for each subplot
for i, v0 in enumerate(categoryValues[0]):
    yLabelText = f"{categorizeBy[0]} = {v0}"
    fig.update_yaxes(title_text=yLabelText, row=i+1, col=1)  # Add y-axis label

for j, v1 in enumerate(categoryValues[1]):
    xLabelText = f"{categorizeBy[1]} = {v1}"
    fig.update_xaxes(title_text=xLabelText, row=num_rows, col=j+1)  # Add x-axis label

fig.show()

In [ ]:
plotdf = plotdf.sort_values(["Params", "houseType"])

categorizeBy = ["C", "roomType"]
categoryValues = [plotdf[category].unique() for category in categorizeBy]
splitAlong = ["WS"]
splitInd = [0, 1]
figures = []
for i, v0 in enumerate(categoryValues[0]):
    for j, v1 in enumerate(categoryValues[1]):
        plotdfSplit = plotdf[((plotdf[categorizeBy[0]] == v0) & (plotdf[categorizeBy[1]] == v1))]
        dfComp, values = getComparativeDf(plotdfSplit, splitAlong)
        # color = indMulti(dfComp.index, ['AofA']).astype(float)
        color = indMulti(dfComp.index, ['delT'])
        fig = px.scatter(x = dfComp['EP_shear'].iloc[:,splitInd[0]] / values[0], y = dfComp['EP_shear'].iloc[:,splitInd[1]] / values[1], 
            color = color, symbol = indMulti(dfComp.index, ['delT']),  hover_data = indMulti(dfComp.index, ['C', 'roomType', 'AofA', 'SS']))
        figures.append(fig)

num_rows = len(categoryValues[0])
num_cols = len(categoryValues[1])

titleText = []
for CValue in categoryValues[1]:
    text = f"{categorizeBy[1]} = "
    if isinstance(CValue, float):
        text = f"{text}{np.round(CValue, 2)}"
    else:
        text = f"{text}{CValue}"
    titleText.append(text)

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)#, subplot_titles=titleText)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        row = int(i / num_cols) + 1  # Calculate row index
        col = (i % num_cols) + 1  # Calculate column index
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800, title_text = f"{splitAlong[0]} {values[splitInd[0]][0]} vs {values[splitInd[1]][0]}")
fig.update_coloraxes(colorscale = "mrybm", colorbar_orientation = "h", colorbar_y = -0.2, cmin = 0, cmax = 360)

fig.add_shape(type="line",
    x0=0, 
    y0=0, 
    x1=np.max(plotdf["EP_shear"]) / 4 * 1.2, 
    y1=np.max(plotdf["EP_shear"]) / 4 * 1.2,
    col = "all",
    row = "all")

# Update x-axis and y-axis properties for each subplot
for i, v0 in enumerate(categoryValues[0]):
    yLabelText = f"{categorizeBy[0]} = {v0}"
    fig.update_yaxes(title_text=yLabelText, row=i+1, col=1)  # Add y-axis label

for j, v1 in enumerate(categoryValues[1]):
    xLabelText = f"{categorizeBy[1]} = {v1}"
    fig.update_xaxes(title_text=xLabelText, row=num_rows, col=j+1)  # Add x-axis label

fig.show()

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf[plotdf["SS"] == True]
plotdf = plotdf.sort_values(["Params", "houseType"])

splitAlong = ["Ri", "C"]
dfComp, values = getComparativeDf(plotdf, splitAlong)
figures = []
baseInd = 0
for i, v in enumerate(values):
    if i == baseInd:
        continue
    fig = px.scatter(x = dfComp['net-mass_flux-Norm'].iloc[:,baseInd], y = dfComp['net-mass_flux-Norm'].iloc[:,i], color = indMulti(dfComp.index, ['WS']), hover_data = indMulti(dfComp.index, ['WS', 'AofA']))
    figures.append(fig)

num_rows = 1
num_cols = int(np.ceil((len(values) - 1) / num_rows))

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        row = int(i / num_cols) + 1  # Calculate row index
        col = (i % num_cols) + 1  # Calculate column index
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800)

fig.add_shape(type="line",
    x0=0, 
    y0=0, 
    x1=np.max(dfComp["net-mass_flux-Norm"]) * 1.2, 
    y1=np.max(dfComp["net-mass_flux-Norm"]) * 1.2,
    col = "all",
    row = "all")

# Update x-axis and y-axis properties for each subplot
labelText = []
for v in values:
    text = ""
    for i, split in enumerate(splitAlong):
        text = f"{text}{split} = {v[i]};  "
    text = text[:-3]
    labelText.append(text)
for i, text in enumerate(labelText):
    if i == baseInd:
        continue
    fig.update_xaxes(title_text=labelText[0], row=1, col=i)  # Add x-axis label
    fig.update_yaxes(title_text=text, row=1, col=i)  # Add y-axis label

fig.show()

In [ ]:
splitAlong = ["Ri", "C"]
# plotdf = plotdf[plotdf["C"] == 2]
dfComp, values = getComparativeDf(plotdf, splitAlong)
figures = []
rows = []
cols = []
for i, v1 in enumerate(values):
    for j, v2 in enumerate(values):
        # if j <= i: # only create plots on or below the diagonal
        rows.append(i+1)
        cols.append(j+1)
        fig = px.scatter(x = dfComp['net-mass_flux-Norm'].iloc[:,j], y = dfComp['net-mass_flux-Norm'].iloc[:,i], color = indMulti(dfComp.index, ['WS']), symbol = indMulti(dfComp.index, ['SS']),  hover_data = indMulti(dfComp.index, ['WS', 'AofA']))
        figures.append(fig)

rows = [r + 1 - min(rows) for r in rows]
num_rows = max(rows)
num_cols = max(cols)

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=rows[i], col=cols[i])
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800)

fig.add_shape(type="line",
    x0=0, 
    y0=0, 
    x1=np.max(dfComp["net-mass_flux-Norm"]) * 1.2, 
    y1=np.max(dfComp["net-mass_flux-Norm"]) * 1.2,
    col = "all",
    row = "all")

# Update x-axis and y-axis properties for each subplot
labelText = []
for v in values:
    text = ""
    for i, split in enumerate(splitAlong):
        text = f"{text}{split} = {np.round(v[i], 2)};  "
    text = text[:-3]
    labelText.append(text)
for i in range(len(labelText)):
    rowInd = num_rows - i
    colInd = i + 1
    fig.update_xaxes(title_text=f'{labelText[i]}', row=num_rows, col=colInd)  # Add x-axis label
    fig.update_yaxes(title_text=f'{labelText[-1-i]}', row=rowInd, col=1)  # Add y-axis label

fig.show()

In [ ]:
figures = []
vMax = 0
vMax = 0
for i, room in enumerate(rooms):
    dfComp, values = getComparativeDf(plotdf[plotdf["roomType"] == room], ["WS"])
    fig = px.scatter(x = dfComp['net-mass_flux-Norm'].iloc[:,0], y = dfComp['net-mass_flux-Norm'].iloc[:,1], color = indMulti(dfComp.index, ['Ri']), hover_data = indMulti(dfComp.index, ['houseType', 'AofA']))
    # fig.show()
    vMax = np.max([vMax, np.max(dfComp["net-mass_flux-Norm"])])
    figures.append(fig)

fig = make_subplots(rows=2, cols=2, subplot_titles=rooms, vertical_spacing=0.2)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        room = rooms[i]
        row = int((i/2)+1)
        col = int((i%2)+1)
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800)

fig.add_shape(type="line",
    x0=0, 
    y0=0, 
    x1=vMax, 
    y1=vMax,
    col = "all",
    row = "all")

fig.show()

In [ ]:
dfCompC, valuesC = getComparativeDf(plotdf, "C")

fig = px.scatter(x = dfCompC['net-mass_flux-Norm'].iloc[:,0], y = dfCompC['net-mass_flux-Norm'].iloc[:,1], symbol = dfCompC["roomType"], color = indMulti(dfCompC.index, ['WS']), hover_data = indMulti(dfCompC.index, ['B', 'AofA']))

fig.add_shape(type="line",
              x0=0, 
              y0=0, 
              x1=np.max(dfCompC["net-mass_flux-Norm"]), 
              y1=np.max(dfCompC["net-mass_flux-Norm"]))

fig.update_layout(
# xaxis_title=r'$\text{Normalized Ventilation Rate} V_{10) = 2 m/s$',
xaxis_title=r'$\text{Buoyancy, Normalized Ventilation Rate } [\frac{Q}{U_{10}A}]$',
yaxis_title=r'$\text{Neutral, Normalized Ventilation Rate } [\frac{Q}{U_{10}A}]$',
# legend_title=r'Room Type'
)

fig.show()
fig.write_image(f'{plotFolder}scatterDensity.png', scale=2)

In [ ]:
fig = px.histogram(
    x=[
        dfCompWS['net-mass_flux-Norm'].iloc[:,0]/dfCompWS['net-mass_flux-Norm'].mean(axis = "columns"), 
        dfCompWS['net-mass_flux-Norm'].iloc[:,1]/dfCompWS['net-mass_flux-Norm'].mean(axis = "columns"),
        dfCompWS['net-mass_flux-Norm'].iloc[:,2]/dfCompWS['net-mass_flux-Norm'].mean(axis = "columns"),
        ],
    histnorm = "percent",
    marginal="box", # or violin, rug
    barmode = "overlay"
    )
fig.show()
fig.write_image(f'{plotFolder}histWS.png', scale=2)

dfCompRi, valuesRi = getComparativeDf(plotdf, "Ri", sort_cols = ['AofA', 'C'])
fig = px.histogram(
    x=[
        dfCompRi['net-mass_flux-Norm'].iloc[:,0]/dfCompRi['net-mass_flux-Norm'].mean(axis = "columns"), 
        dfCompRi['net-mass_flux-Norm'].iloc[:,1]/dfCompRi['net-mass_flux-Norm'].mean(axis = "columns"),
        dfCompRi['net-mass_flux-Norm'].iloc[:,2]/dfCompRi['net-mass_flux-Norm'].mean(axis = "columns"),
        dfCompRi['net-mass_flux-Norm'].iloc[:,3]/dfCompRi['net-mass_flux-Norm'].mean(axis = "columns"),
        ],
    histnorm = "percent",
    marginal="box", # or violin, rug
    barmode = "overlay"
    )
fig.show()
fig.write_image(f'{plotFolder}histRi.png', scale=2)


Scatter plot showing 1 to 1 line
- Ventilation changes due to house location
- Ventilation changes due to wind parameterization
- Ventilation changes due to buoyancy